In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from lifelines import CoxPHFitter
from scipy.stats import mannwhitneyu
import numpy as np
from sksurv.metrics import cumulative_dynamic_auc
from sksurv.metrics import integrated_brier_score
from lifelines import KaplanMeierFitter

## Load and process data

I load the data from the assignment 2. 

In [2]:
# Load data
df_clinical = pd.read_csv('_data/clinical_and_registry.csv').set_index('pnr')
df_clinical['year'] = df_clinical['start'].apply(lambda x: int(x[:4]))
df_clinical['year'] = df_clinical['year']-df_clinical['year'].min()

As the incidence seems to vary in the first assignments, I extracted the year of treatment start to be able to correct for it later. I also kept the variables created in the second assignment for correction. 

I just checks that the different included variables were not too correlated, and thus describing different aspects of the disease. 

In [3]:
display(df_clinical[['crp', 'pain', 'tjc', 'sjc',
       'nb_any_inpatient_visit', 'nb_i_visit', 'year']].corr().round(2))

,crp,pain,tjc,sjc,nb_any_inpatient_visit,nb_i_visit,year
crp,1.00,0.21,0.07,0.25,0.10,0.02,-0.07
pain,0.21,1.00,0.30,0.20,0.06,0.00,0.01
tjc,0.07,0.30,1.00,0.48,-0.05,0.01,-0.03
sjc,0.25,0.20,0.48,1.00,-0.08,0.03,-0.12
nb_any_inpatient_visit,0.10,0.06,-0.05,-0.08,1.00,0.18,-0.05
nb_i_visit,0.02,0.00,0.01,0.03,0.18,1.00,0.04
year,-0.07,0.01,-0.03,-0.12,-0.05,0.04,1.00


In the analysis of the first assignments, we have seen that there were missing data. There are different ways of handling missing data in terms of imputation or modelling. Here I decided to use a model that require data imputation or exclusion of patients with missing data. In an ideal world some discussion with clinicians can help understand if the data are missing at random or not. What I did to check the assumption of missing at random was to compare the distribution between patient with no missing data and patient with at least one missing data. Note that this analysis could be improved but was chosen for the sake of simplicity. I used a Mann Whitney U test to compare the distributions.

In [4]:
df_clinical["nb_missing"] = df_clinical.isna().sum(axis = 1)
dict_res = {}
# For ordial data
for col in ['crp', 'pain', 'tjc', 'sjc','fu_trt', 'nb_any_inpatient_visit', 'nb_i_visit', "year", 'stopped_trt']:
    u, p_value = mannwhitneyu(df_clinical[df_clinical["nb_missing"]==0][col], 
                              df_clinical[df_clinical["nb_missing"]>0][col].dropna())
    u = u/(len(df_clinical[df_clinical["nb_missing"]==0][col])*len(df_clinical[df_clinical["nb_missing"]>0][col].dropna()))
    dict_res[col] = (round(u, 2), round(p_value, 3))

pd.DataFrame(dict_res, index = ['AUC', "p-value"]).T

,AUC,p-value
crp,0.50,0.958
pain,0.46,0.483
tjc,0.49,0.830
sjc,0.52,0.613
fu_trt,0.52,0.297
nb_any_inpatient_visit,0.50,0.848
nb_i_visit,0.50,0.708
year,0.44,0.002
stopped_trt,0.50,0.989


What we see here is that missing values are dependent of the year of inclusion, hopefully this is not the main variable of interest of our study. I will assume here that the inclusion criteria did not change over the years and impute missing data with the mean to try not to loose to much power. 

# Analyse method 

### Separated analysis for treatments:

First, the two treatment might have different impact on the stopping date. Thus instead of including all the interactions with the other variables, I rather separate the analysis for the two treatments, even if it can reduce power. I only keep both treatments when analyzing the effect of the treatment on its interruption.

### Correction:

Then we are interested in the effect of drug, CRP, pain, TJC and SJC, yet to make sure there effect is not related to an other confounding, I also correct for the different variables studied in the other assignments: the tree variable linked to the registries events and the year of treatment start. Note that to limit the decrease of power I did not include interaction between and with correction variables. 

### Model choice:

I used a Cox Proportional Hazard model and check if the value of the proportional hazard was significantly different from 1. Note that I checked the assumptions of the proportional hazard using scaled Schoenfeld residuals computed by the [package](https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html). 

### Model evaluation:

In order to check if the model well describe the data, I decided to compute two metrics:
- __Cumulative Dynamic AUC:__ quantifies how well a model can distinguish subjects who fail by a given time ($t_i\leq t$) from subjects who fail after this time ($t_i > t$) and is suited to right censored data. It captures the same idea as C-index but is a more robust metric. In terms of interpretation, 0.5 correspond to random and 1 to the best outcome. 
- __Integrated Brier Score:__ quantifies the mean squared error of the survival estimated over different time points corrected for right censored data. In terms of interpretation, 0 is the best and 1 the worst. 

Both metrics use several time points, given the distribution of the follow-up time described in the assignment 1, I choose to evaluate the metrics at 10,20,30,40 and 50 months. Note that I have evaluated the models on the train set, evaluating it on a test set would have been more robust but I did not wanted to decrease the power for the association. 

In [5]:
adjustment = "+ nb_any_inpatient_visit + nb_i_visit + year"
features = ['crp', 'pain', 'tjc', 'sjc',
       'nb_any_inpatient_visit', 'nb_i_visit', 'year']
times = [10,20,30,40,50] # Use for model evaluation 

# Drug effect

We have seen that drug A and B might not be given to the same patients, thus we want to adjust for potential confounders. For that different methods exist among which creating pseudo population to compare the treatment groups. I have tried to match B treated patients with A treated patient but as I am not familiar with the different scores, I was not confident with the matching I made and abandoned it. I have thus finally decided to include all possible pairwise adjustment with drug in a cox model. Yet, I am not quite satisfied by the method and if I had had more time I would have rather try to adjust the population using IPTW.

In [6]:
df = df_clinical.copy()
df.loc[:, features] = df[features].fillna(df[features].mean()) # not an issue here that visits columns and year are taken into account as they have no missing values

In [7]:
drug_adjustment = "drug + drug*crp + drug*pain + drug*tjc + drug*sjc + drug*year + drug*nb_any_inpatient_visit + drug*nb_i_visit +"
cph = CoxPHFitter()
cph.fit(df,
        duration_col='fu_trt', event_col='stopped_trt', 
                formula=drug_adjustment + "crp + pain + tjc + sjc + crp * pain + crp * tjc + crp * sjc + pain * tjc + pain * sjc + tjc * sjc " + adjustment)

cph.print_summary()

<lifelines.CoxPHFitter: fitted with 1185 total observations, 644 right-censored observations>
             duration col = 'fu_trt'
                event col = 'stopped_trt'
      baseline estimation = breslow
   number of observations = 1185
number of events observed = 541
   partial log-likelihood = -3588.12
         time fit was run = 2025-04-16 12:50:55 UTC

---
                                  coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                                         
drug[T.B]                        -0.24      0.79      0.34           -0.90            0.42                0.41                1.52
crp                               0.01      1.01      0.01           -0.00            0.02                1.00                1.02
pain                              0.01      1.01      0.00            0.00            0.02                1.00                1.02
tjc                               0.00      1.00      0.03           -0.05            0.06                0.95                1.06
sjc                               0.01      1.01      0.04           -0.06            0.08                0.94                1.08
year                             -0.02      0.98      0.03           -0.08            0.03                0.92                1.03
nb_any_inpatient_visit            0.04      1.04      0.03           -0.02            0.09                0.98                1.10
nb_i_visit                       -0.07      0.93      0.20           -0.46            0.31                0.63                1.37
drug[T.B]:crp                    -0.00      1.00      0.00           -0.01            0.01                0.99                1.01
drug[T.B]:pain                   -0.00      1.00      0.00           -0.01            0.00                0.99                1.00
drug[T.B]:tjc                     0.02      1.02      0.02           -0.02            0.06                0.98                1.06
drug[T.B]:sjc                    -0.03      0.97      0.03           -0.09            0.02                0.92                1.02
drug[T.B]:year                    0.04      1.04      0.05           -0.06            0.14                0.94                1.14
drug[T.B]:nb_any_inpatient_visit -0.05      0.95      0.04           -0.13            0.03                0.88                1.03
drug[T.B]:nb_i_visit              0.17      1.19      0.32           -0.46            0.80                0.63                2.23
crp:pain                         -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
crp:tjc                           0.00      1.00      0.00           -0.00            0.00                1.00                1.00
crp:sjc                           0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pain:tjc                         -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pain:sjc                          0.00      1.00      0.00           -0.00            0.00                1.00                1.00
tjc:sjc                          -0.00      1.00      0.00           -0.00            0.00                1.00                1.00

                                  cmp to     z      p  -log2(p)
covariate                                                      
drug[T.B]                           0.00 -0.72   0.47      1.08
crp                                 0.00  1.36   0.17      2.54
pain                                0.00  3.07 <0.005      8.86
tjc                                 0.00  0.12   0.90      0.15
sjc                                 0.00  0.20   0.84      0.25
year                                0.00 -0.85   0.39      1.34
nb_any_inpatient_visit              0.00  1.26   0.21      2.28
nb_i_visit                          0.00 -

When adjusting for the different confounders, it seems that there are no treatment effect on its stop. I checked the proportional hazard assumption.

In [8]:
cph.check_assumptions(df, p_value_threshold=0.05, show_plots=False)

The ``p_value_threshold`` is set at 0.05. Even under the null hypothesis of no violations, some
covariates will be below the threshold by chance. This is compounded when there are many covariates.
Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and visual tests to determine
the most serious violations. Produce visual plots using ``check_assumptions(..., show_plots=True)``
and looking for non-constant lines. See link [A] below for a full example.



<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 1185 total observations, 644 right-censored observations>
         test_name = proportional_hazard_test

---
                                       test_statistic    p  -log2(p)
crp                              km              2.39 0.12      3.03
                                 rank            2.20 0.14      2.86
crp:pain                         km              2.19 0.14      2.84
                                 rank            2.11 0.15      2.77
crp:sjc                          km              0.10 0.75      0.42
                                 rank            0.12 0.73      0.46
crp:tjc                          km              0.19 0.67      0.58
                                 rank            0.17 0.68      0.56
drug[T.B]                        km              0.14 0.71      0.50
                                 rank            0.15 0.70      0.52
drug[T.B]:crp                    km              0.34 0.56      0.84
                                 rank            0.34 0.56      0.84
drug[T.B]:nb_any_inpatient_visit km              1.96 0.16      2.63
                                 rank            1.81 0.18      2.49
drug[T.B]:nb_i_visit             km              0.02 0.90      0.15
                                 rank            0.02 0.89      0.17
drug[T.B]:pain                   km              2.63 0.10      3.26
                                 rank            2.51 0.11      3.14
drug[T.B]:sjc                    km              0.08 0.78      0.35
                                 rank            0.07 0.80      0.33
drug[T.B]:tjc                    km              0.00 0.95      0.07
                                 rank            0.00 0.97      0.05
drug[T.B]:year                   km              0.01 0.91      0.14
                                 rank            0.00 0.97      0.05
nb_any_inpatient_visit           km              1.50 0.22      2.18
                                 rank            1.52 0.22      2.20
nb_i_visit                       km              0.26 0.61      0.71
                                 rank            0.24 0.63      0.68
pain                             km              2.57 0.11      3.20
                                 rank            2.39 0.12      3.03
pain:sjc                         km              0.00 0.98      0.02
                                 rank            0.00 0.96      0.06
pain:tjc                         km              0.00 0.98      0.03
                                 rank            0.00 0.96      0.06
sjc                              km              0.16 0.69      0.54
                                 rank            0.14 0.71      0.49
tjc                              km              0.00 1.00      0.00
                                 rank            0.01 0.94      0.09
tjc:sjc                          km              0.17 0.68      0.56
                                 rank            0.19 0.66      0.60
year                             km              3.84 0.05      4.32
                                 rank            3.94 0.05      4.41



1. Variable 'year' failed the non-proportional test: p-value is 0.0471.

   Advice 1: the functional form of the variable 'year' might be incorrect. That is, there may be
non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'year' using pd.cut, and then specify it in `strata=['year',
...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


---
[A]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html
[B]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Bin-variable-and-stratify-on-it
[C]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Introduce-time-v

[]

The assumption of proportional hazard is not correct for the year. As it was not the variable of interest, I did not further investigate it, but that might be something to better investigate. 

In [9]:
# Prepare data
train_a = np.core.records.fromarrays(df[['stopped_trt', 'fu_trt']].T.values,
                                       names='col1, col2',
                                       formats='bool, f')
estimate_a = cph.predict_survival_function(df[['drug'] + features], times = times).T

# Compute metrics
print('Cumulative AUC:', round(cumulative_dynamic_auc(train_a, train_a, estimate_a, times, tied_tol=1e-08)[1],2))
print('Integrated Brier Score:', round(integrated_brier_score(train_a, train_a, estimate_a, times),2))

Cumulative AUC: 0.36
Integrated Brier Score: 0.22


Results of the model in terms of event prediction are quite bad. Event ordering metrics (Cumulative AUC) is worst than random and distance is not really good neither. 

## Treatment A

In [10]:
df_a = df_clinical[df_clinical['drug'] == 'A']
df_a.loc[:, features] = df_a[features].fillna(df_a[features].mean()) # not an issue here that visits columns and year are taken into account as they have no missing values

In [11]:
cph = CoxPHFitter()
cph.fit(df_a, duration_col='fu_trt', event_col='stopped_trt', 
                formula="crp + pain + tjc + sjc + crp * pain + crp * tjc + crp * sjc + pain * tjc + pain * sjc + tjc * sjc " + adjustment)

cph.print_summary()

<lifelines.CoxPHFitter: fitted with 687 total observations, 323 right-censored observations>
             duration col = 'fu_trt'
                event col = 'stopped_trt'
      baseline estimation = breslow
   number of observations = 687
number of events observed = 364
   partial log-likelihood = -2215.23
         time fit was run = 2025-04-16 12:50:55 UTC

---
                        coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                               
crp                     0.01      1.01      0.01           -0.00            0.03                1.00                1.03
pain                    0.01      1.01      0.00            0.00            0.02                1.00                1.02
tjc                    -0.00      1.00      0.04           -0.08            0.07                0.93                1.07
sjc                     0.02      1.02      0.04           -0.07            0.10                0.94                1.11
nb_any_inpatient_visit  0.03      1.03      0.03           -0.02            0.09                0.98                1.09
nb_i_visit             -0.07      0.93      0.20           -0.46            0.32                0.63                1.37
year                   -0.03      0.97      0.03           -0.08            0.02                0.92                1.03
crp:pain               -0.00      1.00      0.00           -0.00           -0.00                1.00                1.00
crp:tjc                 0.00      1.00      0.00           -0.00            0.00                1.00                1.00
crp:sjc                 0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pain:tjc                0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pain:sjc               -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
tjc:sjc                -0.00      1.00      0.00           -0.01            0.00                0.99                1.00

                        cmp to     z      p  -log2(p)
covariate                                            
crp                       0.00  1.72   0.08      3.56
pain                      0.00  2.90 <0.005      8.05
tjc                       0.00 -0.12   0.90      0.15
sjc                       0.00  0.40   0.69      0.54
nb_any_inpatient_visit    0.00  1.19   0.23      2.10
nb_i_visit                0.00 -0.36   0.72      0.48
year                      0.00 -1.07   0.29      1.80
crp:pain                  0.00 -2.20   0.03      5.15
crp:tjc                   0.00  0.52   0.60      0.74
crp:sjc                   0.00  1.03   0.30      1.72
pain:tjc                  0.00  0.28   0.78      0.36
pain:sjc                  0.00 -0.04   0.97      0.04
tjc:sjc                   0.00 -1.13   0.26      1.96
---
Concordance = 0.58
Partial AIC = 4456.46
log-likelihood ratio test = 31.50 on 13 df
-log2(p) of ll-ratio test = 8.46

In [12]:
np.exp(cph.confidence_intervals_)

,95% lower-bound,95% upper-bound
covariate,,
crp,0.998140,1.029439
pain,1.004386,1.022943
tjc,0.927047,1.069154
sjc,0.936250,1.105211
nb_any_inpatient_visit,0.978217,1.094851
nb_i_visit,0.630857,1.372396
year,0.919284,1.025176
crp:pain,0.999526,0.999973
crp:tjc,0.999356,1.001116


Some associations were found even if the coefficients are really close to 1: 
- pain: [1.004386,	1.022943] (p-value < 0.005) accelerate the treatment stop,
- crp:pain: [0.999526,	0.999973] (p-value: 0.03) slow the treatment stop.

I checked the proportional hazard assumption.

In [13]:
cph.check_assumptions(df_a, p_value_threshold=0.05, show_plots=False)

The ``p_value_threshold`` is set at 0.05. Even under the null hypothesis of no violations, some
covariates will be below the threshold by chance. This is compounded when there are many covariates.
Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and visual tests to determine
the most serious violations. Produce visual plots using ``check_assumptions(..., show_plots=True)``
and looking for non-constant lines. See link [A] below for a full example.



<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 687 total observations, 323 right-censored observations>
         test_name = proportional_hazard_test

---
                             test_statistic    p  -log2(p)
crp                    km              1.29 0.26      1.97
                       rank            1.22 0.27      1.89
crp:pain               km              1.44 0.23      2.12
                       rank            1.42 0.23      2.10
crp:sjc                km              0.22 0.64      0.64
                       rank            0.21 0.65      0.63
crp:tjc                km              0.52 0.47      1.09
                       rank            0.55 0.46      1.13
nb_any_inpatient_visit km              1.66 0.20      2.34
                       rank            1.68 0.20      2.35
nb_i_visit             km              0.23 0.63      0.66
                       rank            0.22 0.64      0.64
pain                   km              3.13 0.08      3.70
                       rank            3.10 0.08      3.68
pain:sjc               km              0.08 0.78      0.36
                       rank            0.07 0.80      0.33
pain:tjc               km              0.62 0.43      1.21
                       rank            0.61 0.44      1.20
sjc                    km              0.17 0.68      0.56
                       rank            0.15 0.70      0.52
tjc                    km              0.12 0.73      0.45
                       rank            0.11 0.74      0.43
tjc:sjc                km              0.31 0.57      0.80
                       rank            0.34 0.56      0.84
year                   km              4.97 0.03      5.28
                       rank            4.99 0.03      5.30



1. Variable 'year' failed the non-proportional test: p-value is 0.0254.

   Advice 1: the functional form of the variable 'year' might be incorrect. That is, there may be
non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'year' using pd.cut, and then specify it in `strata=['year',
...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


---
[A]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html
[B]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Bin-variable-and-stratify-on-it
[C]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Introduce-time-v

[]

The assumption is not correct for the year. As it was not the variable of interest, I did not further investigate it, but that might be something to better investigate. 

In [14]:
# Prepare data
train_a = np.core.records.fromarrays(df_a[['stopped_trt', 'fu_trt']].T.values,
                                       names='col1, col2',
                                       formats='bool, f')
estimate_a = cph.predict_survival_function(df_a[features], times = times).T

# Compute metrics
print('Cumulative AUC:', round(cumulative_dynamic_auc(train_a, train_a, estimate_a, times, tied_tol=1e-08)[1],2))
print('Integrated Brier Score:', round(integrated_brier_score(train_a, train_a, estimate_a, times),2))

Cumulative AUC: 0.4
Integrated Brier Score: 0.23


Results of the model in terms of event prediction are quite bad. Event ordering metrics (Cumulative AUC) is worst than random and distance is not really good neither. 

## Treatment B

In [15]:
df_b = df_clinical[df_clinical['drug'] == 'B']
df_b.loc[:, features] = df_b[features].fillna(df_b[features].mean())


In [16]:
cph = CoxPHFitter()
cph.fit(df_b, duration_col='fu_trt', event_col='stopped_trt', 
                formula="crp + pain + tjc + sjc + crp * pain + crp * tjc + crp * sjc + pain * tjc + pain * sjc + tjc * sjc " + adjustment)

cph.print_summary()

<lifelines.CoxPHFitter: fitted with 498 total observations, 321 right-censored observations>
             duration col = 'fu_trt'
                event col = 'stopped_trt'
      baseline estimation = breslow
   number of observations = 498
number of events observed = 177
   partial log-likelihood = -1030.24
         time fit was run = 2025-04-16 12:50:56 UTC

---
                        coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                               
crp                    -0.00      1.00      0.01           -0.03            0.02                0.97                1.02
pain                    0.00      1.00      0.01           -0.01            0.02                0.99                1.02
tjc                     0.03      1.03      0.05           -0.06            0.13                0.94                1.14
sjc                    -0.04      0.96      0.06           -0.16            0.08                0.86                1.09
nb_any_inpatient_visit -0.01      0.99      0.03           -0.06            0.04                0.94                1.05
nb_i_visit              0.10      1.11      0.25           -0.39            0.60                0.67                1.82
year                    0.03      1.03      0.04           -0.05            0.12                0.95                1.12
crp:pain                0.00      1.00      0.00           -0.00            0.00                1.00                1.00
crp:tjc                 0.00      1.00      0.00           -0.00            0.00                1.00                1.00
crp:sjc                -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pain:tjc               -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
pain:sjc                0.00      1.00      0.00           -0.00            0.00                1.00                1.00
tjc:sjc                -0.00      1.00      0.00           -0.01            0.00                0.99                1.00

                        cmp to     z    p  -log2(p)
covariate                                          
crp                       0.00 -0.16 0.88      0.19
pain                      0.00  0.60 0.55      0.86
tjc                       0.00  0.70 0.48      1.05
sjc                       0.00 -0.59 0.55      0.85
nb_any_inpatient_visit    0.00 -0.35 0.73      0.46
nb_i_visit                0.00  0.41 0.68      0.55
year                      0.00  0.70 0.48      1.05
crp:pain                  0.00  0.05 0.96      0.06
crp:tjc                   0.00  1.25 0.21      2.23
crp:sjc                   0.00 -0.54 0.59      0.77
pain:tjc                  0.00 -0.55 0.58      0.78
pain:sjc                  0.00  0.66 0.51      0.97
tjc:sjc                   0.00 -0.66 0.51      0.98
---
Concordance = 0.54
Partial AIC = 2086.47
log-likelihood ratio test = 8.31 on 13 df
-log2(p) of ll-ratio test = 0.28

In [17]:
cph.check_assumptions(df_b, p_value_threshold=0.05, show_plots=False)

Proportional hazard assumption looks okay.


[]

No significant results, test without interaction, so I test without interactions.

In [18]:
cph = CoxPHFitter()
cph.fit(df_b.dropna(), duration_col='fu_trt', event_col='stopped_trt', 
                formula="crp + pain + tjc + sjc " + adjustment)

cph.print_summary()

<lifelines.CoxPHFitter: fitted with 498 total observations, 321 right-censored observations>
             duration col = 'fu_trt'
                event col = 'stopped_trt'
      baseline estimation = breslow
   number of observations = 498
number of events observed = 177
   partial log-likelihood = -1031.30
         time fit was run = 2025-04-16 12:50:56 UTC

---
                        coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                               
crp                     0.00      1.00      0.00           -0.01            0.01                0.99                1.01
pain                    0.01      1.01      0.00           -0.00            0.01                1.00                1.01
tjc                     0.02      1.02      0.02           -0.01            0.05                0.99                1.05
sjc                    -0.03      0.97      0.02           -0.07            0.02                0.93                1.02
nb_any_inpatient_visit -0.01      0.99      0.03           -0.06            0.04                0.94                1.04
nb_i_visit              0.10      1.11      0.25           -0.39            0.60                0.68                1.83
year                    0.03      1.03      0.04           -0.05            0.12                0.95                1.13

                        cmp to     z    p  -log2(p)
covariate                                          
crp                       0.00  0.52 0.61      0.72
pain                      0.00  1.45 0.15      2.77
tjc                       0.00  1.10 0.27      1.88
sjc                       0.00 -1.13 0.26      1.95
nb_any_inpatient_visit    0.00 -0.41 0.68      0.56
nb_i_visit                0.00  0.41 0.68      0.56
year                      0.00  0.78 0.44      1.19
---
Concordance = 0.56
Partial AIC = 2076.61
log-likelihood ratio test = 6.17 on 7 df
-log2(p) of ll-ratio test = 0.94

Nothing significant neither. I checked the proportional hazard assumption.

In [19]:
cph.check_assumptions(df_b, p_value_threshold=0.05, show_plots=False)

Proportional hazard assumption looks okay.


[]

In [20]:
# Prepare data
train_b = np.core.records.fromarrays(df_b[['stopped_trt', 'fu_trt']].T.values,
                                       names='col1, col2',
                                       formats='bool, f')
estimate_b = cph.predict_survival_function(df_b[features], times = times).T

# Compute metrics
print('Cumulative AUC:', round(cumulative_dynamic_auc(train_b, train_b, estimate_b, times, tied_tol=1e-08)[1],2))
print('Integrated Brier Score:', round(integrated_brier_score(train_b, train_b, estimate_b, times),2))

Cumulative AUC: 0.4
Integrated Brier Score: 0.2


Results of the model in terms of event description are quite bad. Event ordering metrics (Cumulative AUC) is worst than random and distance is not really good neither. 

# Conclusion

From the analysis, it seems that treatment type has no particular impact on treatment stop. Yet, I am not totally satisfied with the methodology and I rather further investigate. Pain and interaction between pain and crp can help predict the treatment A stop. No variable was found associated with the treatment B stop. Nevertheless, in both cases the model did not very well describe the data. More analysis would be necessary, using other models and if available more covariates. 